# Tutorial: Overview of Quantum Serverless

Structure:

- background, motivation and goals
- building blocks of distributed computation
- resource allocation and execution management
- assembling all together
- electronic structure problem example
- infrastructure
- links and references

## Background, motivation and goals

**Background and motivation**

Today a lot of experiments are happening on a local machines or on manually allocated remote resources. This approach is not scalable and we need to give users a tool to run quantum-classical workloads without worrying about allocating and managing underlying infrastructure.

To achieve the quantum advantage we need to figure out mechanism to combine Classical + Quantum computation, and how orchestrate it.


**Goals**

Create a set of tools that allow user to execute hybrid workloads (combine/orchestrate Classical and Quantum computation).

Efficiently manage abstractions for Quantum and Classical workloads.

Maximizes flexibility for users in multi-cloud and HPC scenarios.

## Building blocks of distributed computation

**Compute resources**

Compute resources can be described as set of computational nodes with resources associated to each of them.
Nodes are performing computation itself. 
Distributed storage is for accessing sharable resources between nodes.

![compute resources](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_compute_resource.png)


In order to resolve distributed compute we need a way to orchestrate our workloads on remote resources.

**Object**

Let's first look how to turn a local object into a remote one.

To turn your local object to distributed one you need to call `put` function. 
It will serialize your object and send it to distributed storage and return you a reference to remote object location. 

![objects diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_put.png)

Example

```python
cirucit = QuantumCircuit(N, M)
...
circuit_reference = quantum_serverless.put(circuit)
```

Now we know how to move local object to remote compute resource. 
Next we need to do something with it. For this we will have remote functions.

**Functions**

As a second step we need to know how to transform our local function into remote one. 

In order to do that you need to annotate function with `run_qiskit_remote` decorator. 
This will turn your function into remote executable.  

![functions diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_function.png)

All you need to do now is call this function and it will be executed as a remote procedure automatically. 

If you run N instances of this function in parallel you need just call it N times in a loop. 
The result of the following code will be a list of references to execution results.

Example

```python
@quantum_serverless.run_qiskit_remote()
def exp_val_remote(cirucit, obs):
    estimator = Estimator(...)
    return estimator.run(circuit, obs)

obs: Operator = ...

exp_val_execution_reference = exp_val_remote(circuit, obs)

circuit_list: List[QuantumCircuit] = [...]

obs_ref = quantum_serverless.put(obs)

exp_val_execution_references = [
    exp_val_remote(circ, obs_ref) 
    for circ in circuit_list
]
```

As you can see we are passing an observable by refernce and circuits by values. Attributes passed by value will be converted to remote objects automatically and resolved within the remote function. We are passing observable by referece because we know it is the same one shared by all parallel function invocations, so we are saving some space. 

**Collecting results**

And of course you need to collect your results back from remote resources. 
In order to do that you need to call the `get` function, which can be applied to object and function references or, alternatively, to a list of references.

![collecting results diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagram_get.png)

Example

```python
collected_circuit = quantum_serverless.get(circuit_reference)
# <QuantumCircuit ...>

collected_exp_value = quantum_serverless.get(exp_val_execution_reference)
# <EstimatorResult ...>
collected_exp_values = quantum_serverless.get(exp_val_execution_references)
# [<EstimatorResult ...>, <EstimatorResult ...>, ...]
```

## Resource allocation and execution management

**Resource allocation**

Some functions are more demanding than others. 
To allocate specific resources to a function, the decorator accepts `target` parameter.
For example, your function may require 2 cpus to be executed. You can pass `target={'cpu': 2}` to decorator and this function will request 2 cpus from your resource capacity to be executed.

![resource allocation diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_resource_allocation.png)

Example

```python
@quantum_serverless.run_qiskit_remote(target={"cpu": 2, "mem": 8})
def exp_val_remote(cirucit, obs):
    estimator = Estimator(...)
    exp_val_result = estimator.run(circuit, obs)
    ...
    return result
```

**Execution management**

Now we figured how to convert our function and objects into remote counterparts and how to request for specific resources.
Next step would be deciding where to run your workloads. 

To have control over location of compute we use configurations in `QuantumServerless` object and python context managers. 

`QuantumServerless` has a notion of `providers` which are abstractions to define where your compute resources are located.

By default you always have a `local` provider which is your local machine where nodes and local cores.
And you can configure as many providers as you want.

![execution management diagram](https://raw.githubusercontent.com/Qiskit-Extensions/quantum-serverless/main/docs/tutorials/images/diagrams_context_management.png)

Example

```python
serverless = QuantumServerless({"providers": [...]})
serverless = QuantumServerless.load_configuration("<PATH_TO_CONFIG_FILE>")
print(serverless)
# <QuantumServerless: providers [local, ibm, ...]>

with serverless.provider("local"):
    exp_val_execution_reference = exp_val_remote(circuit, obs)
    collected_exp_value = quantum_serverless.get(exp_val_execution_reference)

with serverless.provider("ibm"):
    exp_val_execution_reference = exp_val_remote(circuit, obs)
    collected_exp_value = quantum_serverless.get(exp_val_execution_reference)
```

## Assembling all together

Now we have all the important concepts in place and we can assemble our first simple example of a distributed program.

In [4]:
from qiskit.circuit.random import random_circuit
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from quantum_serverless import QuantumServerless, run_qiskit_remote, put, get

serverless = QuantumServerless.load_configuration("./serverless_config.json")

@run_qiskit_remote()
def exp_val_remote(circuit, obs):
    estimator = Estimator()
    return estimator.run(circuit, obs).result()

circuit_list = [random_circuit(2, 2) for _ in range(3)]
obs = SparsePauliOp(["IZ"])

with serverless:
    obs_ref = put(obs)
    exp_val_execution_references = [
        exp_val_remote(circ, obs_ref) 
        for circ in circuit_list
    ]
    collected_exp_values = get(exp_val_execution_references)
    
print(collected_exp_values)

[EstimatorResult(values=array([1.]), metadata=[{}]), EstimatorResult(values=array([1.]), metadata=[{}]), EstimatorResult(values=array([-0.8821732]), metadata=[{}])]
